# Climb gradient

This section describes climb gradient analysis for the example airplane. This analysis is performed to determine whether the airplane satisfies the climb gradient requirements or not. It is difficult to compute maximum climb gradient at any given condition for a propeller-driven airplane. Hence, in this section, a simple analysis method is used, similar to the method used in [constraint analysis](../constraint_analysis/climb.ipynb). The climb gradient (CGR) is computed as (equation 3.29, Roskam Part 1)

$$
    CGR = \frac{18.97\eta_p\sqrt{\sigma}}{W/P \sqrt{W/S}} - \frac{1}{L/D},
$$  

where $P$ is the power, $W$ is the weight, and $\eta_p$ is propeller efficiency. The $S$ is the wing reference area and $\sigma$ is the ratio of the density at  given altitude to sea-level. Note that this is the same equation used in constraint analysis. However, parameters in this equation are computed using better methods. The best possible climb gradient occurs when $C_L$ is close to its maximum value. This can be dangerous since the aircraft will be flying close to stall speed, and hence, a margin of 0.2 is used as recommended in Roskam Part 1. The $L/D$ can be computed using 

$$
    \frac{L}{D} = \frac{C_L}{C_{D_0} + \dfrac{C_L^2}{\pi Ae}}.
$$

The following sub-sections compute CGR for the three conditions - all engine operative (AEO), one engine inoperative (OEI), and blaked landing (BL).

Below code block imports required packages and defines a function for computing CGR:

In [1]:
import numpy as np

def compute_CGR(W, CL, CD0, sigma, e, P):

    A = 8 # aspect ratio
    S = 134 # sq ft
    prop_eff = 0.8
    K = 1/np.pi/A/e
    L_by_D = CL / ( CD0 + K*CL**2 )

    CGR = 18.97 * prop_eff * (sigma*CL)**0.5 / (W/P) / (W/S)**0.5 - 1/L_by_D

    return CGR

## AEO

The flight conditions for AEO case is given in [constraint analysis](../constraint_analysis/climb.ipynb) section. Below code computes CGR for AEO case:

In [2]:
MTOW = 5374 # lbs
W = 0.984 * MTOW # lbs
CLmax = 1.8
CL = CLmax - 0.2 # 0.2 is safety margin
CD0 = 0.03363 + 0.024011 # TO conditions
rho_sealevel = 0.00237717 # slugs/cu ft, sea-level
sigma = 1
e = 0.81 - 0.05 # TO condition
P_max = 2*298 # hp, max to power
P_max_cont = P_max / 1.1 # hp, max continuous power

CGR = compute_CGR(W, CL, CD0, sigma, e, P_max_cont)

print(f"Climb gradient for AEO case: {CGR:.4f}")

Climb gradient for AEO case: 0.1933


Note that the computed AEO climb gradient is greater than the required CGR of 0.083.

## OEI

The flight conditions for OEI case is given in [constraint analysis](../constraint_analysis/climb.ipynb) section. Note that drag due to stopped propeller should be taken into account. Below code computes CGR for OEI case:

In [3]:
W = 0.95 * MTOW # lbs
CLmax = 1.44
CL = CLmax - 0.2 # 0.2 is safety margin
CD0 = 0.03363 # clean conditions
rho_5000 = 0.00204834 # slugs/cu ft, at 5000 ft
sigma = rho_5000 / rho_sealevel
e = 0.81 # clean condition
P = P_max/2 * (sigma - (1-sigma)/7.75) # power at 5000 ft

# drag due to stopped propeller
S = 134 # sq ft
dia = 5.8 # ft
Aprop = np.pi * (dia)**2 / 4
CD0_prop = 0.1 * sigma * Aprop / S

CGR = compute_CGR(W, CL, CD0+CD0_prop, sigma, e, P)

print(f"Climb gradient for OEI case: {CGR:.4f}")

Climb gradient for OEI case: 0.0234


Note that the computed OEI climb gradient is greater than the required CGR of 0.015.

## Blaked landing

The flight conditions for blaked landing case is given in [constraint analysis](../constraint_analysis/climb.ipynb) section. Below code computes CGR for blaked landing case:

In [5]:
W = 0.95 * MTOW # lbs
CLmax = 2.2
CL = CLmax - 0.2 # 0.2 is safety margin
CD0 = 0.03363 + 0.048074 # landing conditions
sigma = 1
e = 0.81 - 0.1 # landing condition

CGR = compute_CGR(W, CL, CD0, sigma, e, P_max)

print(f"Climb gradient for AEO case: {CGR:.4f}")

Climb gradient for AEO case: 0.2530


Note that the computed blaked landing climb gradient is greater than the required CGR of 0.03. This concludes the climb gradient analysis.